<a href="https://colab.research.google.com/github/shabirdewji/DHT/blob/master/shabir_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
# import tflearn
# import tensorflow
import random
import json
import pickle

with open("shabir.json") as file:
    data = json.load(file)
    print('JSON FILE SHABIR LOADED')
# try:
#     with open("data.pickle", "rb") as f:
#         words, labels, training, output = pickle.load(f)
# except:
    words  = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w.lower()) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
                
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)
        # print("\n",w,"============training========")
        # print(training)
        # print("\n",w,"============output========")
        # print(output)

    # training = numpy.array(training)
    
    # print("\n",w,"============training after numpy========")
    # print(training)
    
    output = np.array(output)
    # print("\n",w,"============output after numpy========")
    # print(output)
    
    # with open("data.pickle", "wb") as f:
    #     pickle.dump((words, labels, training, output), f)
    
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return bag

def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        # inp = 'where have you lived'
        # print('input = ',inp)
        if inp.lower() == "quit":
            break
        # results = model.predict([bag_of_words(inp, words)])
        results = ([bag_of_words(inp, words)])
        # print("\n",words,"============results========")
        # print(results)
        # bag = results 
        i=0
        bg = [0 for _ in range(len(words))]
        # print('bg ',bg)
        
        for myList in [results]:
            for item in myList:
                for itt in item:
                    bg[i] = itt
                    # print('bg',i)
                    i += 1
        # print('bg ',bg)
        
        no_intents = 0
        no_patterns = 0
        for intent in data["intents"]:
            for pattern in intent["patterns"]:
                no_patterns += 1
            no_intents += 1
            
        score=[]
        w=0
        w1=no_patterns
        while w < w1 :
            score.append(0)
            w += 1
        # print('number of data intents', no_intents)
        # print('number of data pattern', no_patterns)  
        # print('score',score)
        # print("\n","==========training========")
        k=0
        for t in training:
            j=0
            for i in t:
                # print('t:',t,'i',i,'j',j)
                if bg[j]==i  and i==1:
                    score[k] += 1
                j += 1
            k += 1

#data["intents"] has  all the info
#data intents {'tag': 'citys', 'patterns': ['where have you lived', 'cities lived'], 'responses': ['I have lived in Mombasa, Nairobi, Glasgow, London, Torornto, Regina and Calgary']}
#data intents {'tag': 'zachary', 'patterns': ['who is My son', "son's name"], 'responses': ['zachary']}          

        # for tg in data["intents"]:
        #     print('data intents',tg )
        
        # print('score',score)
            
        results_index = np.argmax(score)
#this returns the index of the highest score
#docs_x [['where', 'have', 'you', 'lived'], ['cities', 'lived'], ['who', 'is', 'My', 'son'], ['son', "'s", 'name']]
#docs_y ['citys', 'citys', 'zachary', 'zachary']
        tag = docs_y[results_index]
        tag2= docs_x[results_index]
        sc = score[results_index]
        # print('results_index: ',results_index)
        # print('labels',labels)

        # print('Picked Phrase: ',tag2, 'Picked tag: ',tag,'Picked score: ',sc)
        
#the labels are the tags
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        print(random.choice(responses))

chat()

JSON FILE SHABIR LOADED
Start talking with the bot (type quit to stop)!
You: Mary
Maryam
You: who was the ex-wife
Mary
You: quit
